# Recurrent Neural Network - Random name generator

Congratulations! You’re expecting a baby! Whether it’s a boy or a girl, you want your child’s name to be unique and very special. What you do is to go online and visit a whole bunch of "best baby names" list, or use random name pickers. However, these names have been used over and over by at least a few hundred others before. If this is going through your mind, then keep reading!

The solution that I am offering here is to use deep learning. First, you should collect a list of few hundred favorite names like the ones I've included here (IranianNames.txt). To generate new "cool" names, we will build a character-level recurrent neural network (RNN) and train it on the dataset. After our language model learned the patterns, we ask it to generate names you have never seen before!

Great! Let's begin by writing some functions.
 

In [19]:
import numpy as np
import matplotlib as plt

## 1 - Loading and preprocessing data

The following cell loads the dataset of Persian names and creates a list of unique characters. As you can see the total number of characters is 27 (a-z plus "\n" newline character).

In [21]:
with open('IranianNames.txt', 'r') as datafile :
    names = datafile.read().lower()

chars = list(set(names))

num_names, vocab_size = len(names), len(chars)
print('total number of names: %d, total number of unique characters: %d' % (num_names, vocab_size))

total number of names: 8750, total number of unique characters: 27


Now we create one hash table (ch2idx, see cell below) to map each character (["\n", a-z]) to an index from 0-26, and a second hash table (idx2ch, see cell below) that maps each index back to its corresponding character. This will help us figure out what index corresponds to what character in the probability distribution output of the softmax layer. 

In [22]:
ch2idx = { ch:idx for idx, ch in enumerate(sorted(chars)) }
idx2ch = { idx:ch for idx, ch in enumerate(sorted(chars)) }
print('char to index: ', ch2idx)
print('index to char: ', idx2ch)

char to index:  {'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
index to char:  {0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


## 2 - Implementing useful functions

In this section we are implementing some useful functions. 

In [24]:
def initialize_parameters (n_x, n_h, n_y):
    
    """
    Inputs :
        n_x: input layer size
        n_h: hidden layer size
        n_y: output layer size
    Outputs:
        W_hx: Weight matrix from input to hidden layer, shape (n_h, n_x)
        b_hx: Bias from input to hidden layer, shape (n_h, 1)
        W_hh: Weight matrix from first hidden layer to second hidden layer, shape (n_h, n_h)
        W_yh: Weight matrix from the second hidden layer to output layer, shape (n_y, n_h)
        b_yh: Bias from the second hidden layer to the output layer, shape (n_y, 1)
    """
    W_hx = np.random.randn(n_h, n_x)*0.01 # input to hidden
    b_hx = np.zeros((n_h, 1)) # hidden bias
    W_hh = np.random.randn(n_h, n_h)*0.01 # hidden to hidden
    W_yh = np.random.randn(n_y, n_h)*0.01 # hidden to output
    b_yh = np.zeros((n_y, 1)) # output bias
        
    return W_hx, b_hx, W_hh, W_yh, b_yh

In [26]:
## softmax function
def softmax (x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum(axis=0)

In [25]:
def rnn_step_forward (a_prev, x, W_hx, b_hx, W_hh, W_yh, b_yh):
    """
    Inputs :
        x : input vector, shape (n_x, 1)
        a_prev: previous hidden state, shape, (n_h, 1)
        W_hx: Weight matrix from input to hidden layer
        b_hx: Bias from input to hidden layer
        W_hh: Weight matrix from first hidden layer to second hidden layer
        W_yh: Weight matrix from the second hidden layer to output layer
        b_yh: Bias from the second hidden layer to the output layer
    Outputs:
        a_next: next hidden state, shape (n_h, 1)
        p_t: unnormalized log probabilities for next characters, shape (n_y, 1)
    """
    a_next = np.tanh(np.dot(W_hx, x) + np.dot(W_hh, a_prev) + b_hx) 
    p_t = softmax(np.dot(W_yh, a_next) + b_yh)  
    
    return a_next, p_t

In [27]:
def rnn_forward (X, Y, a0, W_hx, b_hx, W_hh, W_yh, b_yh, vocab_size=27):
    """
    Inputs :
        X: input matrix, shape (n_x, m)
        Y: label vector, shape (m, 1)
        a0: initial hidden state, shape (n_h, 1)
        W_hx: Weight matrix from input to hidden layer
        b_hx: Bias from input to hidden layer
        W_hh: Weight matrix from first hidden layer to second hidden layer
        W_yh: Weight matrix from the second hidden layer to output layer
        b_yh: Bias from the second hidden layer to the output layer
        vocab_size: number of vocabs, scalar-default is 27
    Outputs:
        cache tuple that includes:
            y_hat: predicted labels, shape (m, 1)
            a: hidden state dictionary
            x: one-hot representation of X elements
        loss: loss value, scalar
    """
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    loss = 0 # Initialize the loss
    for t in range(len(X)):
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(a[t-1], x[t], W_hx, b_hx, W_hh, W_yh, b_yh)
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)

    return loss, cache

In [47]:
def rnn_step_backward (dy, grads, W_hx, b_hx, W_hh, W_yh, b_yh, x, a, a_prev):
    """
    Inputs :
        dy: label error vector
        grads: gradient dictionary
        W_hx: Weight matrix from input to hidden layer
        b_hx: Bias from input to hidden layer
        W_hh: Weight matrix from first hidden layer to second hidden layer
        W_yh: Weight matrix from the second hidden layer to output layer
        b_yh: Bias from the second hidden layer to the output layer
        x: one-hot representation of characters
        a: hidden state, shape, (n_h, 1)
        a_prev: previous hidden state, shape (n_h, 1)
    Outputs:
        grads: gradient dictionary
    """
    grads['dWyh'] += np.dot(dy, a.T)
    grads['dbyh'] += dy
    da = np.dot(W_yh.T, dy) + grads['da_next'] # backpropagation into h
    dummy = (1 - a * a) * da # backpropagation through tanh nonlinearity
    grads['dbhx'] += dummy
    grads['dWhx'] += np.dot(dummy, x.T)
    grads['dWhh'] += np.dot(dummy, a_prev.T)
    grads['da_next'] = np.dot(W_hh.T, dummy)
    return grads

In [48]:
def rnn_backward(X, Y, W_hx, b_hx, W_hh, W_yh, b_yh, cache):
    """
    Inputs :
        X: input matrix, shape (n_x, m)
        Y: label vector, shape (m, 1)
        a0: initial hidden state, shape, (n_h, 1)
        W_hx: Weight matrix from input to hidden layer
        b_hx: Bias from input to hidden layer
        W_hh: Weight matrix from first hidden layer to second hidden layer
        W_yh: Weight matrix from the second hidden layer to output layer
        b_yh: Bias from the second hidden layer to the output layer
        cache tuple that includes:
            y_hat: predicted labels
            a: hidden state dictionary
            x: one-hot representation of X elements
    Outputs:
        grads: gradient dictionary
        a: hidden state dictionary
    """
    grads = {} # Initialize gradients as an empty dictionary
    (y_hat, a, x) = cache # Retrieve from cache
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    grads['dWhx'], grads['dWhh'], grads['dWyh'] = np.zeros_like(W_hx), np.zeros_like(W_hh), np.zeros_like(W_yh)
    grads['dbhx'], grads['dbyh'] = np.zeros_like(b_hx), np.zeros_like(b_yh)
    grads['da_next'] = np.zeros_like(a[0])
    
    for t in reversed(range(len(X))): # reverse propagation through time
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        grads = rnn_step_backward(dy, grads, W_hx, b_hx, W_hh, W_yh, b_yh, x[t], a[t], a[t-1])
    
    return grads, a

In [32]:
def update_parameters(W_hx, b_hx, W_hh, W_yh, b_yh, grads, lr):

    W_hx += -lr * grads['dWhx']
    W_hh += -lr * grads['dWhh']
    W_yh += -lr * grads['dWyh']
    b_hx  += -lr * grads['dbhx']
    b_yh  += -lr * grads['dbyh']
    
    return W_hx, b_hx, W_hh, W_yh, b_yh

In the following cell, we will implement a function to prevent exploding of gradients and keep it between some range -N and N. 

In [33]:
def clip(grads, clip_val):
    '''    
    Inputs:
        grads: a dictionary of the gradients "dWhh", "dWhx", "dWyh", "dbhx", "dbyh"
        clip_val: gradients will be clipped between -clip_val and clip_val
    Outputs:
        grads: the dictionary of the gradients
    '''
    dWhh, dWhx, dWyh, dbhx, dbyh = grads['dWhh'], grads['dWhx'], grads['dWyh'], grads['dbhx'], grads['dbyh']
   
    for item in [dWhx, dWhh, dWyh, dbhx, dbyh]:
        np.clip(item, -clip_val, clip_val, out=item)
    
    grads = {"dWhh": dWhh, "dWhx": dWhx, "dWyh": dWyh, "dbhx": dbhx, "dbyh": dbyh}
    
    return grads

## 3 - Building the language model 

Now let's build our character-level language model for text generation. The following function performs only one iteration of stochastic gradient descent (SGD) with clipped gradients according to the following steps:

- Forward propagation through the RNN to compute the loss
- Backward propagation through time to compute the gradients
- Clip the gradients if necessary 
- Update synaptic weights using gradient descent 

In [50]:
def optimize(X, Y, a_prev, W_hx, b_hx, W_hh, W_yh, b_yh, lr=0.01):
    """    
    Inputs :
        X: input matrix, shape (n_x, m)
        Y: label vector, shape (m, 1)
        a_prev: previous hidden state, shape, (n_h, 1)
        W_hx: Weight matrix from input to hidden layer
        b_hx: Bias from input to hidden layer
        W_hh: Weight matrix from first hidden layer to second hidden layer
        W_yh: Weight matrix from the second hidden layer to output layer
        b_yh: Bias from the second hidden layer to the output layer
        learning_rate: learning rate, default 0.01
    Outputs:
        loss: value of the loss function (cross-entropy)
        grads: a dictionary containing: dWhx, dWhh, dWyh, dbhx, dbyh
        a: the last hidden state
    """
    loss, cache = rnn_forward(X, Y, a_prev, W_hx, b_hx, W_hh, W_yh, b_yh, vocab_size=27) # Forward propagation
    grads, a = rnn_backward(X, Y, W_hx, b_hx, W_hh, W_yh, b_yh, cache) # Backpropagation
    grads = clip(grads, 10) # clip gradients between -10 and 10
    W_hx, b_hx, W_hh, W_yh, b_yh = update_parameters(W_hx, b_hx, W_hh, W_yh, b_yh, grads, lr) # Update parameters
        
    return loss, grads, a[len(X)-1]

## 4 - Training the model

Now we train the model on the collected names. It is very important to shuffle the dataset, so that stochastic gradient descent visits the examples in random order. 
 

In [89]:
def model(fname, idx2ch, ch2idx, num_iterations=100000, n_h=40, vocab_size=27):
    """
    Inputs:
        fname: name of the file
        idx2ch: dictionary that maps the index to a character
        ch2idx: dictionary that maps a character to an index
        num_iterations: number of iterations to train the model for
        n_h: number of hidden units of the RNN cell
        vocab_size: size of the vocabulary
    Outputs:
        W_hx: Weight matrix from input to hidden layer, shape (n_h, n_x)
        b_hx: Bias from input to hidden layer, shape (n_h, 1)
        W_hh: Weight matrix from first hidden layer to second hidden layer, shape (n_h, n_h)
        W_yh: Weight matrix from the second hidden layer to output layer, shape (n_y, n_h)
        b_yh: Bias from the second hidden layer to the output layer, shape (n_y, 1)
    """
    n_x, n_y = vocab_size, vocab_size # Retrieve n_x and n_y from vocab_size    
    W_hx, b_hx, W_hh, W_yh, b_yh = initialize_parameters(n_x, n_h, n_y) # Initialize parameters
    
    loss = -np.log(1.0/vocab_size) # Initialize loss
    
    with open(fname) as f:
        names = f.readlines()
    names = [x.lower().strip() for x in names]
    np.random.shuffle(names) # Shuffle list of all names
    
    a_prev = np.zeros((n_h, 1)) # Initialize the hidden state
    for ii in range(1, num_iterations):
        idx = ii % len(names)
        Y = [ch2idx[ch] for ch in names[idx]] + [char_to_ix["\n"]]
        X = [None] + Y[:-1]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        curr_loss, grads, a_prev = optimize(X, Y, a_prev, W_hx, b_hx, W_hh, W_yh, b_yh, lr=0.01)
                
        loss = loss * 0.999 + curr_loss * 0.001 # smoothing loss

        if ii % 5000 == 0:
            print('Iteration: %d, Loss: %f' % (ii, loss) + '\n')
        
    return W_hx, b_hx, W_hh, W_yh, b_yh

In [90]:
W_hx, b_hx, W_hh, W_yh, b_yh = model('IranianNames.txt', idx2ch, ch2idx, num_iterations=100000)

Iteration: 5000, Loss: 16.805458

Iteration: 10000, Loss: 15.376390

Iteration: 15000, Loss: 14.865807

Iteration: 20000, Loss: 14.553933

Iteration: 25000, Loss: 14.312119

Iteration: 30000, Loss: 14.160799

Iteration: 35000, Loss: 13.946571

Iteration: 40000, Loss: 14.014759

Iteration: 45000, Loss: 13.966208

Iteration: 50000, Loss: 13.732016

Iteration: 55000, Loss: 13.735051

Iteration: 60000, Loss: 13.584354

Iteration: 65000, Loss: 13.637023

Iteration: 70000, Loss: 13.684620

Iteration: 75000, Loss: 13.404673

Iteration: 80000, Loss: 13.865663

Iteration: 85000, Loss: 13.386959

Iteration: 90000, Loss: 13.557052

Iteration: 95000, Loss: 13.366332



## 5 - Sampling

Now thw model is trained and it is able to generate new patterns. First, we feed the network a vector of zeros and carry out one step of forward propagation. Then, we randomly pick (see the next function) the next character's index according to the probability distribution specified by $\hat{y}$. Then, the network is fed with picked index to generate another index until either the newline character is selected or this loop reaches a maximum value ($maxLen$).

In [91]:
def sample(W_hx, b_hx, W_hh, W_yh, b_yh, ch2idx, seed, maxLen=20):
    """
    Inputs:
        W_hx: Weight matrix from input to hidden layer, shape (n_h, n_x)
        b_hx: Bias from input to hidden layer, shape (n_h, 1)
        W_hh: Weight matrix from first hidden layer to second hidden layer, shape (n_h, n_h)
        W_yh: Weight matrix from the second hidden layer to output layer, shape (n_y, n_h)
        b_yh: Bias from the second hidden layer to the output layer, shape (n_y, 1)
        ch2idx: dictionary that maps a character to an index
        maxLen: maximum length of generated names
    Outputs:
        indices: a list containing the indices of the sampled characters
    """
    vocab_size = b_yh.shape[0]
    n_h = W_hh.shape[1]
    
    x = np.zeros((vocab_size, 1)) # one-hot vector of zeros as the initial input
    a_prev = np.zeros((n_h, 1)) # initialize a previous hidden state
    
    indices = []
    idx = -1 
    newline_character = ch2idx['\n']   
    counter = 0
    while (idx!=newline_character and counter!=maxLen):
        
        # Forward propagation
        a = np.tanh(np.dot(W_hx, x) + np.dot(W_hh, a_prev) + b_hx)
        y = softmax(np.dot(W_yh, a) + b_yh)
        
        # Sample the index of a character within the vocabulary from the probability distribution y
        np.random.seed(seed)
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
        
        indices.append(idx) # Append the index to "indices"
        
        # Updating the input character with the sampled index.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a # Update "a_prev" to be "a"
        seed += 1
        counter +=1
        
    if (counter==maxLen) :
        indices.append(char_to_ix['\n'])
    
    return indices

In [94]:
n = 50
seed = 0
for ii in range(n):
    # Sample indices and print them
    indices = sample(W_hx, b_hx, W_hh, W_yh, b_yh, ch2idx, seed)
    seed += 1
    txt = ''.join(idx2ch[idx] for idx in indices)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

Jambodobe
Ginube
Goudi
Joosbi
Tasboar
Bodoar
Sasana
Ayara
Sana
Asakhar
Ohanan
Ainou
Ashid
Pareeh
Izare
Sale
Bahanahatah
Bidi
Maha
Aralaza
Kamiyl
Alitl
Anuz
Izladeh
Shahrourb
Seiroude
Behrokha
Gorooz
Mipand
Shardad
Marhanel
Bolahen
Sabanlbila
Bahpour
Anozan
Goudan
Mongofe
Shahilarshad
Firooqey
Jalabele
Ghadiza
Bahrye
Farzib
Avve
Ssii
Yila
Pabandir
Aalehan
Anousha
Behroo


## Conclusions

You can see that the language model proposed here is very easy to implement while it is very good in generating meaningful patterns of characters. At the beginning of training, it was generating random patterns, but after training it generates some good names. However, not all the names are cool or meaningful. Please feel free to tweak parameters to see if you can get even better names.

Obviously, the larger and more diverse training dataset would lead to more meaningful and novel patterns through improvisation. Training on a small dataset, like the one we used here, can cause the model to overfit which leads to generating many of the previously seen names and limited improvisation. 

**References**:

- This notebook is inspired from Sequence Model course taugh by Andrew Ng: https://www.coursera.org/learn/nlp-sequence-models. To learn more about text generation, also check out Karpathy's [blog post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/).